# Δημιουργήστε ένα μοντέλο ταξινόμησης: Νόστιμες Ασιατικές και Ινδικές Κουζίνες


## Ταξινομητές κουζίνας 2

Σε αυτό το δεύτερο μάθημα ταξινόμησης, θα εξερευνήσουμε `περισσότερους τρόπους` για να ταξινομήσουμε κατηγορικά δεδομένα. Θα μάθουμε επίσης τις συνέπειες της επιλογής ενός ταξινομητή αντί για έναν άλλο.

### [**Κουίζ πριν το μάθημα**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Προαπαιτούμενα**

Υποθέτουμε ότι έχετε ολοκληρώσει τα προηγούμενα μαθήματα, καθώς θα συνεχίσουμε με ορισμένες έννοιες που μάθαμε προηγουμένως.

Για αυτό το μάθημα, θα χρειαστούμε τα εξής πακέτα:

-   `tidyverse`: Το [tidyverse](https://www.tidyverse.org/) είναι μια [συλλογή πακέτων R](https://www.tidyverse.org/packages) σχεδιασμένη για να κάνει την επιστήμη δεδομένων πιο γρήγορη, εύκολη και διασκεδαστική!

-   `tidymodels`: Το [tidymodels](https://www.tidymodels.org/) είναι ένα [πλαίσιο συλλογής πακέτων](https://www.tidymodels.org/packages/) για μοντελοποίηση και μηχανική μάθηση.

-   `themis`: Το [πακέτο themis](https://themis.tidymodels.org/) παρέχει επιπλέον βήματα συνταγών για την αντιμετώπιση μη ισορροπημένων δεδομένων.

Μπορείτε να τα εγκαταστήσετε ως εξής:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Εναλλακτικά, το παρακάτω script ελέγχει αν έχετε τα απαραίτητα πακέτα για την ολοκλήρωση αυτής της ενότητας και τα εγκαθιστά για εσάς σε περίπτωση που λείπουν.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Ένας χάρτης ταξινόμησης**

Στο [προηγούμενο μάθημα](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), προσπαθήσαμε να απαντήσουμε στο ερώτημα: πώς επιλέγουμε ανάμεσα σε πολλαπλά μοντέλα; Σε μεγάλο βαθμό, αυτό εξαρτάται από τα χαρακτηριστικά των δεδομένων και τον τύπο του προβλήματος που θέλουμε να λύσουμε (για παράδειγμα ταξινόμηση ή παλινδρόμηση).

Προηγουμένως, μάθαμε για τις διάφορες επιλογές που έχετε όταν ταξινομείτε δεδομένα χρησιμοποιώντας το cheat sheet της Microsoft. Το πλαίσιο Μηχανικής Μάθησης της Python, Scikit-learn, προσφέρει ένα παρόμοιο αλλά πιο λεπτομερές cheat sheet που μπορεί να σας βοηθήσει περαιτέρω να περιορίσετε τους εκτιμητές σας (ένας άλλος όρος για τους ταξινομητές):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Συμβουλή: [επισκεφθείτε αυτόν τον χάρτη online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) και κάντε κλικ κατά μήκος της διαδρομής για να διαβάσετε την τεκμηρίωση.
>
> Ο [ιστότοπος αναφοράς Tidymodels](https://www.tidymodels.org/find/parsnip/#models) παρέχει επίσης εξαιρετική τεκμηρίωση για διαφορετικούς τύπους μοντέλων.

### **Το σχέδιο** 🗺️

Αυτός ο χάρτης είναι πολύ χρήσιμος μόλις αποκτήσετε μια ξεκάθαρη κατανόηση των δεδομένων σας, καθώς μπορείτε να "περπατήσετε" κατά μήκος των διαδρομών του για να φτάσετε σε μια απόφαση:

-   Έχουμε \>50 δείγματα

-   Θέλουμε να προβλέψουμε μια κατηγορία

-   Έχουμε δεδομένα με ετικέτες

-   Έχουμε λιγότερα από 100.000 δείγματα

-   ✨ Μπορούμε να επιλέξουμε ένα Linear SVC

-   Αν αυτό δεν λειτουργήσει, δεδομένου ότι έχουμε αριθμητικά δεδομένα

    -   Μπορούμε να δοκιμάσουμε έναν ✨ KNeighbors Classifier

        -   Αν και αυτό δεν λειτουργήσει, δοκιμάστε ✨ SVC και ✨ Ensemble Classifiers

Αυτή είναι μια πολύ χρήσιμη διαδρομή για να ακολουθήσετε. Τώρα, ας ξεκινήσουμε χρησιμοποιώντας το [tidymodels](https://www.tidymodels.org/) πλαίσιο μοντελοποίησης: μια συνεκτική και ευέλικτη συλλογή πακέτων R που αναπτύχθηκαν για να ενθαρρύνουν την καλή στατιστική πρακτική 😊.

## 2. Διαχωρισμός των δεδομένων και αντιμετώπιση μη ισορροπημένων συνόλων δεδομένων.

Από τα προηγούμενα μαθήματα, μάθαμε ότι υπήρχε ένα σύνολο κοινών συστατικών στις κουζίνες μας. Επίσης, υπήρχε μια αρκετά άνιση κατανομή στον αριθμό των κουζινών.

Θα τα αντιμετωπίσουμε αυτά με τους εξής τρόπους:

-   Αφαιρώντας τα πιο κοινά συστατικά που δημιουργούν σύγχυση μεταξύ διαφορετικών κουζινών, χρησιμοποιώντας `dplyr::select()`.

-   Χρησιμοποιώντας μια `recipe` που προεπεξεργάζεται τα δεδομένα ώστε να είναι έτοιμα για μοντελοποίηση, εφαρμόζοντας έναν αλγόριθμο `over-sampling`.

Έχουμε ήδη δει τα παραπάνω στο προηγούμενο μάθημα, οπότε αυτό θα είναι παιχνιδάκι 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Αντιμετώπιση μη ισορροπημένων δεδομένων

Τα μη ισορροπημένα δεδομένα συχνά έχουν αρνητικές επιπτώσεις στην απόδοση του μοντέλου. Πολλά μοντέλα αποδίδουν καλύτερα όταν ο αριθμός των παρατηρήσεων είναι ίσος και, ως εκ τούτου, δυσκολεύονται με μη ισορροπημένα δεδομένα.

Υπάρχουν κυρίως δύο τρόποι αντιμετώπισης μη ισορροπημένων συνόλων δεδομένων:

-   προσθήκη παρατηρήσεων στην κατηγορία μειοψηφίας: `Over-sampling`, π.χ. χρησιμοποιώντας έναν αλγόριθμο SMOTE, ο οποίος συνθετικά δημιουργεί νέα παραδείγματα της κατηγορίας μειοψηφίας χρησιμοποιώντας τους πλησιέστερους γείτονες αυτών των περιπτώσεων.

-   αφαίρεση παρατηρήσεων από την κατηγορία πλειοψηφίας: `Under-sampling`

Στο προηγούμενο μάθημά μας, δείξαμε πώς να αντιμετωπίσουμε μη ισορροπημένα σύνολα δεδομένων χρησιμοποιώντας μια `recipe`. Μια recipe μπορεί να θεωρηθεί ως ένα σχέδιο που περιγράφει ποια βήματα πρέπει να εφαρμοστούν σε ένα σύνολο δεδομένων για να είναι έτοιμο για ανάλυση δεδομένων. Στην περίπτωσή μας, θέλουμε να έχουμε ίση κατανομή στον αριθμό των κουζινών μας για το `training set` μας. Ας ξεκινήσουμε!


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Τώρα είμαστε έτοιμοι να εκπαιδεύσουμε μοντέλα 👩‍💻👨‍💻!

## 3. Πέρα από τα μοντέλα πολυωνυμικής παλινδρόμησης

Στο προηγούμενο μάθημα, εξετάσαμε μοντέλα πολυωνυμικής παλινδρόμησης. Ας εξερευνήσουμε μερικά πιο ευέλικτα μοντέλα για ταξινόμηση.

### Μηχανές Υποστήριξης Διανυσμάτων (Support Vector Machines)

Στο πλαίσιο της ταξινόμησης, οι `Μηχανές Υποστήριξης Διανυσμάτων` είναι μια τεχνική μηχανικής μάθησης που προσπαθεί να βρει ένα *υπερεπίπεδο* που "καλύτερα" διαχωρίζει τις κατηγορίες. Ας δούμε ένα απλό παράδειγμα:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ δεν διαχωρίζει τις κλάσεις. H2~ τις διαχωρίζει, αλλά μόνο με μικρό περιθώριο. H3~ τις διαχωρίζει με το μέγιστο περιθώριο.

#### Γραμμικός Ταξινομητής Υποστηρικτικών Διανυσμάτων

Η ομαδοποίηση Υποστηρικτικών Διανυσμάτων (SVC) είναι μέλος της οικογένειας τεχνικών μηχανικής μάθησης Υποστηρικτικών Διανυσμάτων. Στο SVC, το υπερεπίπεδο επιλέγεται ώστε να διαχωρίζει σωστά `την πλειοψηφία` των παρατηρήσεων εκπαίδευσης, αλλά `μπορεί να ταξινομήσει λανθασμένα` κάποιες παρατηρήσεις. Επιτρέποντας σε ορισμένα σημεία να βρίσκονται στη λάθος πλευρά, το SVM γίνεται πιο ανθεκτικό σε ακραίες τιμές και συνεπώς καλύτερο στη γενίκευση σε νέα δεδομένα. Η παράμετρος που ρυθμίζει αυτή την παραβίαση αναφέρεται ως `cost`, η οποία έχει προεπιλεγμένη τιμή 1 (δείτε `help("svm_poly")`).

Ας δημιουργήσουμε έναν γραμμικό SVC ορίζοντας `degree = 1` σε ένα πολυωνυμικό μοντέλο SVM.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Τώρα που έχουμε καταγράψει τα βήματα προεπεξεργασίας και την προδιαγραφή του μοντέλου σε μια *ροή εργασίας*, μπορούμε να προχωρήσουμε στην εκπαίδευση του γραμμικού SVC και να αξιολογήσουμε τα αποτελέσματα ταυτόχρονα. Για τις μετρικές απόδοσης, ας δημιουργήσουμε ένα σύνολο μετρικών που θα αξιολογεί: `accuracy`, `sensitivity`, `Positive Predicted Value` και `F Measure`.

> Το `augment()` θα προσθέσει στήλη(-ες) για προβλέψεις στα δεδομένα που δίνονται.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Υποστήριξη Διανυσματικής Μηχανής

Η υποστήριξη διανυσματικής μηχανής (SVM) είναι μια επέκταση του υποστηρικτικού διανυσματικού ταξινομητή, ώστε να μπορεί να διαχειριστεί ένα μη γραμμικό όριο μεταξύ των κατηγοριών. Στην ουσία, οι SVMs χρησιμοποιούν το *kernel trick* για να διευρύνουν τον χώρο χαρακτηριστικών, προσαρμοζόμενοι σε μη γραμμικές σχέσεις μεταξύ των κατηγοριών. Μία δημοφιλής και εξαιρετικά ευέλικτη συνάρτηση πυρήνα που χρησιμοποιείται από τους SVMs είναι η *Radial basis function.* Ας δούμε πώς θα αποδώσει στα δεδομένα μας.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Πολύ καλύτερα 🤩!

> ✅ Παρακαλώ δείτε:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> για περαιτέρω ανάγνωση.

### Ταξινομητές Πλησιέστερου Γείτονα

Το *K*-nearest neighbor (KNN) είναι ένας αλγόριθμος στον οποίο κάθε παρατήρηση προβλέπεται με βάση την *ομοιότητά* της με άλλες παρατηρήσεις.

Ας εφαρμόσουμε έναν στο σύνολο δεδομένων μας.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Φαίνεται ότι αυτό το μοντέλο δεν αποδίδει τόσο καλά. Πιθανότατα η αλλαγή των παραμέτρων του μοντέλου (δείτε `help("nearest_neighbor")`) θα βελτιώσει την απόδοση του μοντέλου. Φροντίστε να το δοκιμάσετε.

> ✅ Παρακαλώ δείτε:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> για να μάθετε περισσότερα σχετικά με τους ταξινομητές *K*-Nearest Neighbors.

### Ταξινομητές Ensemble

Οι αλγόριθμοι Ensemble λειτουργούν συνδυάζοντας πολλούς βασικούς εκτιμητές για να παράγουν ένα βέλτιστο μοντέλο είτε μέσω:

`bagging`: εφαρμογή μιας *συνάρτησης μέσου όρου* σε μια συλλογή βασικών μοντέλων

`boosting`: δημιουργία μιας ακολουθίας μοντέλων που βασίζονται το ένα στο άλλο για να βελτιώσουν την προβλεπτική απόδοση.

Ας ξεκινήσουμε δοκιμάζοντας ένα μοντέλο Random Forest, το οποίο δημιουργεί μια μεγάλη συλλογή δέντρων αποφάσεων και στη συνέχεια εφαρμόζει μια συνάρτηση μέσου όρου για ένα καλύτερο συνολικό μοντέλο.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Μπράβο 👏!

Ας πειραματιστούμε επίσης με ένα μοντέλο Boosted Tree.

Το Boosted Tree ορίζει μια μέθοδο συνόλου που δημιουργεί μια σειρά από διαδοχικά δέντρα αποφάσεων, όπου κάθε δέντρο εξαρτάται από τα αποτελέσματα των προηγούμενων δέντρων, σε μια προσπάθεια να μειώσει σταδιακά το σφάλμα. Εστιάζει στα βάρη των αντικειμένων που ταξινομήθηκαν λανθασμένα και προσαρμόζει την εφαρμογή για τον επόμενο ταξινομητή ώστε να διορθώσει.

Υπάρχουν διαφορετικοί τρόποι για να προσαρμόσετε αυτό το μοντέλο (δείτε `help("boost_tree")`). Σε αυτό το παράδειγμα, θα προσαρμόσουμε τα Boosted trees μέσω της μηχανής `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Παρακαλώ δείτε:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Εξετάζει το μοντέλο AdaBoost, το οποίο αποτελεί μια καλή εναλλακτική στο xgboost.
>
> για να μάθετε περισσότερα σχετικά με τους Ensemble classifiers.

## 4. Επιπλέον - σύγκριση πολλαπλών μοντέλων

Έχουμε εφαρμόσει αρκετά μοντέλα σε αυτό το εργαστήριο 🙌. Μπορεί να γίνει κουραστικό ή δύσκολο να δημιουργήσουμε πολλούς workflows από διαφορετικά σύνολα προεπεξεργαστών και/ή προδιαγραφές μοντέλων και στη συνέχεια να υπολογίσουμε τις μετρικές απόδοσης μία προς μία.

Ας δούμε αν μπορούμε να αντιμετωπίσουμε αυτό το ζήτημα δημιουργώντας μια συνάρτηση που εφαρμόζει μια λίστα από workflows στο εκπαιδευτικό σύνολο και στη συνέχεια επιστρέφει τις μετρικές απόδοσης βάσει του συνόλου δοκιμών. Θα χρησιμοποιήσουμε τις `map()` και `map_dfr()` από το πακέτο [purrr](https://purrr.tidyverse.org/) για να εφαρμόσουμε συναρτήσεις σε κάθε στοιχείο της λίστας.

> Οι συναρτήσεις [`map()`](https://purrr.tidyverse.org/reference/map.html) σας επιτρέπουν να αντικαταστήσετε πολλούς βρόχους for με κώδικα που είναι πιο συνοπτικός και ευκολότερος στην ανάγνωση. Το καλύτερο μέρος για να μάθετε σχετικά με τις συναρτήσεις [`map()`](https://purrr.tidyverse.org/reference/map.html) είναι το [κεφάλαιο για την επανάληψη](http://r4ds.had.co.nz/iteration.html) στο R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


Το πακέτο [**workflowset**](https://workflowsets.tidymodels.org/) επιτρέπει στους χρήστες να δημιουργούν και να εφαρμόζουν εύκολα έναν μεγάλο αριθμό μοντέλων, αλλά έχει σχεδιαστεί κυρίως για να λειτουργεί με τεχνικές επαναδειγματοληψίας όπως η `διασταυρούμενη επικύρωση`, μια προσέγγιση που δεν έχουμε καλύψει ακόμα.

## **🚀Πρόκληση**

Καθεμία από αυτές τις τεχνικές έχει έναν μεγάλο αριθμό παραμέτρων που μπορείτε να προσαρμόσετε, όπως για παράδειγμα το `cost` στα SVMs, το `neighbors` στο KNN, το `mtry` (Τυχαία Επιλεγμένοι Προγνωστικοί Παράγοντες) στο Random Forest.

Ερευνήστε τις προεπιλεγμένες παραμέτρους του καθενός και σκεφτείτε τι θα σήμαινε η προσαρμογή αυτών των παραμέτρων για την ποιότητα του μοντέλου.

Για να μάθετε περισσότερα σχετικά με ένα συγκεκριμένο μοντέλο και τις παραμέτρους του, χρησιμοποιήστε: `help("model")`, π.χ. `help("rand_forest")`.

> Στην πράξη, συνήθως *εκτιμούμε* τις *καλύτερες τιμές* αυτών των παραμέτρων εκπαιδεύοντας πολλά μοντέλα σε ένα `προσομοιωμένο σύνολο δεδομένων` και μετρώντας πόσο καλά αποδίδουν όλα αυτά τα μοντέλα. Αυτή η διαδικασία ονομάζεται **βελτιστοποίηση**.

### [**Κουίζ μετά το μάθημα**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Ανασκόπηση & Αυτομελέτη**

Υπάρχει αρκετή ορολογία σε αυτά τα μαθήματα, οπότε αφιερώστε λίγο χρόνο για να αναθεωρήσετε [αυτήν τη λίστα](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) χρήσιμων όρων!

#### ΕΥΧΑΡΙΣΤΟΥΜΕ:

[`Allison Horst`](https://twitter.com/allison_horst/) για τη δημιουργία των καταπληκτικών εικονογραφήσεων που κάνουν τη R πιο φιλόξενη και ενδιαφέρουσα. Βρείτε περισσότερες εικονογραφήσεις στη [συλλογή της](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) και [Jen Looper](https://www.twitter.com/jenlooper) για τη δημιουργία της αρχικής έκδοσης αυτού του module σε Python ♥️

Καλή Μάθηση,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Artwork by @allison_horst</figcaption>



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
